In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder

In [5]:
data_dir = r'F:\plant disease dataset'
df=pd.read_csv(r'F:\plant disease dataset\images.csv')
df.head()

,filepath,Disease
0,F:\plant disease dataset\archive\New Plant Dis...,Apple___Apple_scab
1,F:\plant disease dataset\archive\New Plant Dis...,Apple___Apple_scab
2,F:\plant disease dataset\archive\New Plant Dis...,Apple___Apple_scab
3,F:\plant disease dataset\archive\New Plant Dis...,Apple___Apple_scab
4,F:\plant disease dataset\archive\New Plant Dis...,Apple___Apple_scab


In [6]:
df.iloc[0]['filepath']

'F:\\plant disease dataset\\archive\\New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\train\\Apple___Apple_scab\\00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG'

In [8]:
df.iloc[0]['Disease']

'Apple___Apple_scab'

In [9]:
class CropDiseaseDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = f"{self.df.iloc[idx]['filepath']}"

        label = self.df.iloc[idx]['Disease']

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.long)
        return image, label

In [10]:
class CropDiseaseDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.batch_size = batch_size

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),#data augmentation
            transforms.RandomRotation(10),#data augmentation<<
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
        ])

    def setup(self, stage=None):
        df = pd.read_csv(r'F:\plant disease dataset\images.csv')
        df['Disease'] = LabelEncoder().fit_transform(df['Disease'])
        num_classes = len(df['Disease'].value_counts())
        print("Number of classes: {}".format(num_classes))
        train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

        self.train_dataset = CropDiseaseDataset(
            train_df,
            transform=self.transform
        )

        self.val_dataset = CropDiseaseDataset(
            val_df,
            transform=self.transform
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
        )

In [11]:
data_module = CropDiseaseDataModule(
    batch_size=64,
)

data_module.setup()

Number of classes: 38


In [12]:
class CropDiseaseClassifier(pl.LightningModule):
    def __init__(self, num_classes=38, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32), #means ekta batch e jotogulo image thakbe sobgulo k tader avg and std diye oi image gulo k normalize kore fela
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),

            nn.Flatten(),

            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(0.5),#inactive the 50% weights(randomly) on the layer to avoid overfitting
            nn.Linear(512, num_classes)
        )

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()

        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()

        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(#this is learning rate scheduler.This basically does
            optimizer,
            mode='min',
            factor=0.1,
            patience=5
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss'
            }
        }

In [13]:
model = CropDiseaseClassifier(num_classes=38)

In [ ]:
trainer = pl.Trainer(
    max_epochs=2,
)

trainer.fit(model, data_module)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


Number of classes: 38


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name      ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model     │ Sequential       │ 26.1 M │ train │     0 │
│ 1 │ criterion │ CrossEntropyLoss │      0 │ train │     0 │
└───┴───────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 26.1 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 26.1 M                                                                                               
Total estimated model params size (MB): 104                                                                        
Modules in train mode: 23                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment 
analysis)\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 'val_dataloader' 
does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 
to `num_workers=7` in the `DataLoader` to improve performance.

C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment 
analysis)\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:434: The 
'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 
`num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

C:\Users\User\Downloads\60 days of python\day-38(Aspect base sentiment analysis)\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
trainer.validate(model, data_module)